In [2]:
import torch
import torch.nn as nn

In [ ]:
x = torch.randn(10,5)  # input tensor
w = torch.randn(5, 3, requires_grad=True)

print(x.shape, w.shape)
print(x,w)

In [ ]:
#matrix multiplication
y1 = x@w
print(y1.shape)
print(y1)
y2 = x.matmul(w)
print(y2)
y3 = torch.matmul(x, w)
print(y3)

In [ ]:
# matrix multiplication
linear = torch.nn.Linear(5, 3, bias=False)
print(linear.weight)
print(linear.bias)
linear.weight.data = w.T
print(w)
print(linear.weight)
y = linear(x)
print(y)
print(y3)

In [ ]:
# element wise multiplcation

print(w)

z = w * w

print(z)

i = 3
j = 2
print(w[i][j]**2)

In [ ]:
print(w, w.dtype, w.device)
w1 = w.bfloat16()
print(w1.dtype)

In [ ]:
m1 = nn.Linear(20, 30)
m2 = nn.Linear(30, 40)
x = torch.randn(128, 20)
y1 = m1(x)
y2 = m2(y1)
print(y2.shape)

In [ ]:
x_large = torch.randn(128, 4096,30, 20)
y1_large = m1(x_large)
y2_large = m2(y1_large)
print(y2.shape)

In [ ]:
x_large = torch.randn(128, 4096,30, 20)
b, s, h, w = x_large.shape
print(b, s, h, w)
x_large = x_large.view(b * s * h, w)
print(x_large.shape)

y1 = m1(x_large)
y2 = m2(y1)
print(y2.shape)
y2 = y2.view(b, s, h, 40)
print(y2.shape)


In [ ]:
x = torch.ones((1,1,5,5))
print(x)
x[:,:,:,0]=0
x[:,:,:,1]=0
x[:,:,0,:]=0
x[:,:,1,:]=0
print(x)
conv_w = torch.ones((1,1,3,3))

y = nn.functional.conv2d(x, conv_w)
print(y)

In [12]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [16]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 16*4*4)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = LeNet()

In [17]:
# loss function

ce_loss = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        l = loss_fn(pred, y)
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = l.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct



epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, ce_loss, optimizer)
    test(test_dataloader, model, ce_loss)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307752  [    0/60000]
loss: 2.311440  [ 6400/60000]
loss: 2.314778  [12800/60000]
loss: 2.305566  [19200/60000]
loss: 2.312666  [25600/60000]
loss: 2.300375  [32000/60000]
loss: 2.293721  [38400/60000]
loss: 2.297170  [44800/60000]
loss: 2.303488  [51200/60000]
loss: 2.284255  [57600/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.300549 

Epoch 2
-------------------------------
loss: 2.303244  [    0/60000]
loss: 2.306667  [ 6400/60000]
loss: 2.309264  [12800/60000]
loss: 2.302010  [19200/60000]
loss: 2.307486  [25600/60000]
loss: 2.296831  [32000/60000]
loss: 2.290577  [38400/60000]
loss: 2.293575  [44800/60000]
loss: 2.299700  [51200/60000]
loss: 2.281992  [57600/60000]
Test Error: 
 Accuracy: 10.6%, Avg loss: 2.296716 

Epoch 3
-------------------------------
loss: 2.299142  [    0/60000]
loss: 2.302365  [ 6400/60000]
loss: 2.303915  [12800/60000]
loss: 2.298060  [19200/60000]
loss: 2.301935  [25600/60000]
loss: 2.292897  [32000/600